In [2]:

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
data_df = pd.read_csv("Speed Dating Data.csv", encoding="ISO-8859-1")
data_df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [59]:
fields = data_df.columns
num_dates_per_male = data_df[data_df.gender == 1].groupby('iid').apply(len)
num_dates_per_female = data_df[data_df.gender == 0].groupby('iid').apply(len)

# Preprocessing
def str_to_float(series):
    return series.apply(lambda x: str(x).replace(",", "")).astype('float64')

for trait in ['mn_sat', 'tuition', 'income']:
    data_df[trait] = str_to_float(data_df[trait])
data_df['pid'] = data_df['pid'].fillna(-1.0).astype('int64')  # Invalid PID as -1

# Compute the features for additional traits
def standardize_feature(series):
    return (series - series.mean()) / series.std(ddof=0)
    
    
# PROFILE OF THE PERSON
data_df['financial'] = standardize_feature(data_df['tuition']) \
                       .add(standardize_feature(data_df['income']), fill_value=0.0)
#importance bet/ date and go_out
a=5
b=1
data_df['experience'] = a*data_df['date'] + b*data_df['imprelig']
data_df['experience'] = standardize_feature(data_df['experience'])
data_df['int'] = standardize_feature(data_df['mn_sat']);

# divide them into female data and male data
data_df_m = data_df[data_df.gender == 1] #data of male
data_df_f = data_df[data_df.gender == 0] #data of female


# Create a dataframe containing information for each person that needs to be looked up
# profiles = data_df[['iid', 'mn_sat', 'goal', 'field_cd', 'financial', 'experience']]\
#            .set_index(keys='iid').drop_duplicates()
    
profiles_m = data_df_m[['iid', 'int',  'field_cd', 'financial', 'experience']]\
           .set_index(keys='iid').drop_duplicates()
profiles_m = profiles_m.fillna(profiles_m.mean())  # Fill NaN values with mean
data_df_m = data_df_m.fillna(data_df_m.mean())

profiles_f = data_df_f[['iid', 'int',  'field_cd', 'financial', 'experience']]\
           .set_index(keys='iid').drop_duplicates()
profiles_f = profiles_f.fillna(profiles_f.mean())  # Fill NaN values with mean
data_df_f = data_df_f.fillna(data_df_f.mean())

# profiles: the basic features of a person

In [27]:
# some clustering test
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin


df_train = profiles_m.as_matrix()
df_train = df_train[:,[0,2,3]]

k_means = KMeans(n_clusters=6, random_state=0).fit(df_train)

k_means.cluster_centers_

In [63]:
data_df_m=data_df_m.fillna(profiles_m.mean()) 


# Computing trait similarities/differences
data_df_m['age_diff'] = data_df_m['age'].sub(data_df_m['age_o']).abs()  # Age difference

def is_similar_profession(x, profiles):
    if np.isnan(x['field_cd']) or np.isnan(x['pid']) or x['pid'] not in profiles.index:
        return False
    else:
        return x['field_cd'] == profiles.loc[x['pid']]['field_cd']
data_df_m['sim_profession'] = data_df_m[['field_cd', 'pid']]\
                            .apply(lambda x: is_similar_profession(x, profiles), axis=1)

def trait_difference(trait):
    trait_other = data_df_m['pid'].apply(lambda x: profiles.loc[x][trait] if x in profiles.index else None)
    return data_df_m[trait].sub(trait_other)
    
for trait in ['mn_sat', 'goal', 'experience', 'financial']:
    data_df_m[trait + '_diff'] = trait_difference(trait)


KeyError: 'mn_sat'

In [64]:
data_df_m['mn_sat_diff']

KeyError: 'mn_sat_diff'

In [72]:
data_df_m['mn_sat' + '_diff'] = trait_difference('mn_sat')

KeyError: 'mn_sat'

In [71]:
data_df_m["mn_sat"]

100     1321.272656
101     1321.272656
102     1321.272656
103     1321.272656
104     1321.272656
105     1321.272656
106     1321.272656
107     1321.272656
108     1321.272656
109     1321.272656
110     1321.272656
111     1321.272656
112     1321.272656
113     1321.272656
114     1321.272656
115     1321.272656
116     1321.272656
117     1321.272656
118     1321.272656
119     1321.272656
120     1321.272656
121     1321.272656
122     1321.272656
123     1321.272656
124     1321.272656
125     1321.272656
126     1321.272656
127     1321.272656
128     1321.272656
129     1321.272656
           ...     
8348    1400.000000
8349    1400.000000
8350    1400.000000
8351    1400.000000
8352    1400.000000
8353    1400.000000
8354    1400.000000
8355    1400.000000
8356    1321.272656
8357    1321.272656
8358    1321.272656
8359    1321.272656
8360    1321.272656
8361    1321.272656
8362    1321.272656
8363    1321.272656
8364    1321.272656
8365    1321.272656
8366    1321.272656
